### Apply pre-processing steps to Sentinel-2 images

In [ ]:
img_folder = '/mnt/d/a2autocls2023/imgmad/ano2021/'
ofolder    = '/mnt/d/a2autocls2023/imgmad/proc2021/'

cliprst = '/mnt/d/a2autocls2023/ref/rst_madeira.tif'

In [ ]:
import os
from glass.cons.stl import get_ibands, get_lwibands
from glass.pys.oss import lst_ff, mkdir, fprop, copy_file
from glass.pys.tm import now_as_str
from glass.dtr.stl  import unzip_img
from glass.rst.rmp import match_cellsize_and_clip
from glass.rst.mos import rsts_to_mosaic

In [ ]:
ibands = get_ibands()
lwbands = get_lwibands()

_ibands = {ibands[i] : lwbands[i] for i in range(len(ibands))}

In [ ]:
_ibands

In [ ]:
imgs = lst_ff(img_folder, file_format='.zip')

In [ ]:
# Image pre-processing

_bands = []
tifs_f = mkdir(os.path.join(ofolder, 'tifs'))

for img in imgs:
    # Create folder for temp files
    zfolder = mkdir(os.path.join(ofolder, now_as_str()))
    
    # Unzip images
    bands = unzip_img(img, zfolder)
    
    # Match cellsize and clip
    bands = match_cellsize_and_clip(
        [bands[b] for b in bands],
        bands["B02_10m"], tifs_f,
        isint=True, clipgeo=cliprst,
        ws=zfolder
    )

In [ ]:
# Go for mosaic

# ID tiles and days

tifs = lst_ff(tifs_f, file_format='.tif')

mosaics = {}

for img in tifs:
    name = fprop(img, 'fn')
    
    t, d, b, r = name.split('_')
    _b = f"{b}_{r}"
    _d = d.split('T')[0]
    
    if _d not in mosaics:
        mosaics[_d] = {}
        
    if _ibands[_b] not in mosaics[_d]:
        mosaics[_d][_ibands[_b]] = [img]
    
    else:
        mosaics[_d][_ibands[_b]].append(img)

# Create mosaics for each day

for day in mosaics:
    for band in mosaics[day]:
        mos = os.path.join(ofolder, f"{band}_{day}.tif")
        if len(mosaics[day][band]) < 2:
            nb = copy_file(mosaics[day][band][0], mos)
        
        else:
            nb = rsts_to_mosaic(
                mosaics[day][band], mos,
                api='rasterio', method='max'
            )
        
        mosaics[day][band] = nb